# Classification by Logistic Regression

We've studied data. We've visualized high dimensional spaces projected onto dimensions which maximized the variance of the data. We've shaped and reshaped matrixes to get them ready to feed into a model. Today, we're going to classify our first set of data.

## Logistic Regression

[sklearn.linear_model.LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

Logistic regression is a machine learning model that, once trained, estimates the probability that an input value `x` meets some "requirement", or doesn't. In logistic regression, `x` is a numerical data-matrix, as we are so familiar with, and `y` is a biary label, or class (that is, it is either 0 or 1).


## Configuration

Maximum likelihood estimation: assuming that:

$P(y=1|x) = \frac{1}{(1 + e^{-(x^Tw)})}$

then the parameters that maximize the likelihood function are found through gradient descent updates. 

For each sample, for each $x_i$:

$w_i = w_i + \alpha(y_i - p)x_i$

The first iteration can start with random valies for $w_i$, or they can all begin at $0$.

Logistic Regression has at least eight algorithmic solutions, many of which are supported by `sklearn`. Different hyper parameters apply to different algorithms, such as:

* learning_rate (alpha)
* epochs
* starting weights
* tolerance for stopping criteria (that is, stop when error < t)

### Implementation

See [Logistic Regression](http://www.stat.cmu.edu/~cshalizi/uADA/12/lectures/ch12.pdf)
and [A comparison of numerical optimizers for logistic regression](https://tminka.github.io/papers/logreg/minka-logreg.pdf)

## Training

Before we train a Logistic Regression model, it is important to separate the features and labels into a training and testing set:

```
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state=42)
```

Example code:
   
    regr = linear_model.LogisticRegression()
    regr.fit(x_train, y_train)


## Classification

Models like Logistic Regression are designed to create predictions based on new values:

Example code:
    
    prediction = regr.predict(x)
    
However, in research practice, models are used to compare their benchmark performance, typically by measuring their test set accuracy.


## Testing / Validation

For Linear Regression, we "tested on our training set", which is a big no-no in machine learning. On higher-powered ML models it is pretty easy to get 100% accuracy when testing on our training set. When a model is perfectly tuned to a single set of data this is called "overfitting", and it specifically doesn't tell us about how the model will perform on any previously unseen data.

```
# Training accuracy:
training_accuracy = np.sum(y_hat_train==y_train)/len(y_hat_train)

# Test accuracy:
y_hat_test = regr.predict(x_test)
test_accuracy = np.sum(y_hat_test==y_test)/len(y_hat_test)
`````

## Measuring results 

Many of the metrics used in regression are not relevant or useful for evaluating the performance of classification models.

### RMSE

Root Mean Squared Error is relevant for regression of scalar values, not classes. RMSE measures the average error of each sample in the same scale as the samples themselves. When compared with the variance of the data, informs the experimenter of the quality of the model. If $RMSE < \sigma$, that is, the RMSE is less than the standard deviation of the data, then each prediction is high quality, predicting a value close to what it might actually be. The larger the ratio $\frac{RMSE}{\sigma}$, the lower-quality the prediction.

### MSE

Mean squared error is related to variance the same way that RMSE is related to $\sigma$. Used during error calculation because the relative scale of the error is not relevant, only its minimization. This decreases computation time and frequently makes calculating partial derivatives much easier.

### SSE

Sum of squared-errors is equal to $n \times MSE$. SSE is useful in a binary classification problem, as the ratio $\frac{SSE}{n}$ is the percent error rate. Furthermore, since the labels are binary values, the SSE simply gives the number of misclassified points, or the number of errors $e$. It is not useful in the multi-class classification problem. Accuracy, a common metric used in the evaluation of models is related to the percent error rate, as it is the proportion of points classified correctly.

$e = \sum{(\hat{y}-y)^2}$

$error = \frac{e}{n}$
   
$accuracy = 1 - error$


# Your Assignment

Use `sklearn.linear_model.LogisticRegression` to identify survivors of the titanic dataset. We've already prepared titanic. Now separate it into features `x` and the label `survived = y`. Using LogisticRegression, train a model to predict `y` from `x`.

* Try converting the categorical features in titanic into OneHot features. Can you improve results? [One Hot Features](https://yashuseth.blog/2017/12/14/how-to-one-hot-encode-categorical-variables-of-a-large-dataset-in-python/)
* Try creating a few "feature cross" columns, where you create a new feature in titanic data that is equal to the product of two other features, ie $x_{new} = x_j \times x_j$ [Feature Crosses](https://developers.google.com/machine-learning/crash-course/feature-crosses/video-lecture)

# Thinking about your assignment

1. __What am I being asked to do?__
Perform classification on the Titanic dataset, using its features to predict the binary outcome, `Survived`.


2. __What coding steps need to be taken to satisfy the problem?__
    * Load and preprocess the data, encoding the categorical features so that they can be fed into a logistic regression model. 
    * Split the data into training and testing data
    * Select features, and engineer some feature crosses.
    * Train logistic regression models on different sets of features, comparing their performance on the test set using a metric such as accuracy.

3. __What must I do to claim that I have "completed" the assignment?__
    * Train multiple logistic regression models, and report their accuracy
    * Have at least a couple models trained on sets of features which include feature crosses

In [0]:
# Complete assignment here.
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from itertools import chain, combinations

## Loading, Preprocessing and Encoding Data

In [2]:
def fill_mixed_median_mode(dataframe, medians=list()):
    """ Fill missing values with median for specified column, otherwise mode
    
    Args:
        dataframe (pandas.core.frame.DataFrame): rows of observations of features
        medians (list): columns to fill missing values with median instead of mode
        
    Returns:
        dataframe with no missing values
    """
    
    
    null = dataframe.isnull().any()
    null_cols = list(null[null].index)
    
    fill = pd.Series([data[c].median() if c in medians else data[c].mode()[0]
                     for c in null_cols], index=null_cols)
    
    dataframe[null_cols] = dataframe[null_cols].fillna(fill)
    return dataframe

data = sns.load_dataset('titanic')
data = data.drop(['alive','adult_male','who','class','embark_town', 'deck'], axis=1)
data = data.drop_duplicates()

data_f = fill_mixed_median_mode(data, ['age', 'fare'])

for label in ['embarked','sex', 'alone']:
    data_f[label] = LabelEncoder().fit_transform(data_f[label])

embarked_one_hot = OneHotEncoder().fit_transform(data_f[['embarked']]).toarray()
embarked = pd.DataFrame(embarked_one_hot, 
                        columns=['Southampton', 'Cherbourg', 'Queenstown'], 
                        dtype=np.int64)

data_f = data_f.reset_index(drop=True)
data_enc = data_f.join([embarked])
data_enc = data_enc.drop(['embarked'], axis=1)

data_enc.head()

,survived,pclass,sex,age,sibsp,parch,fare,alone,Southampton,Cherbourg,Queenstown
0,0,3,1,22.0,1,0,7.2500,0,0,0,1
1,1,1,0,38.0,1,0,71.2833,0,1,0,0
2,1,3,0,26.0,0,0,7.9250,1,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,0,1
4,0,3,1,35.0,0,0,8.0500,1,0,0,1


## Train-Test Split

I will drop one of the one-hot features because it is multicollinear with the other two one-hot features. I chose to only encode `Embarked` with one-hot encodings because the other categorical features have what seems to be a meaningful way to speak of ordering.

In [0]:
x_train, x_test, y_train, y_test = train_test_split(
    data_enc.drop(['survived', 'Southampton'], axis=1), 
    data_enc[['survived']], test_size=0.3, random_state=41
)

## Comparing Models on Different Sets of Features

In [0]:
def compare_models(feature_sets, x_train, x_test, y_train, y_test, log=100):
    models = pd.DataFrame([])
    
    for ix, features in enumerate(feature_sets):
        features = list(features)
        if len(features) == 0:
            continue
        
        X_train = x_train[features].values.reshape(-1, len(features))
        X_test = x_test[features].values.reshape(-1, len(features))

        model = LogisticRegression()
        model.fit(X_train, y_train.values.ravel())
        yhat = model.predict(X_test)

        sse = np.sum(np.power(yhat-y_test.values.ravel(), 2))
        error = sse / yhat.shape[0]
        accuracy = 1 - error
        
        params = model.coef_[0]
        intercept, = model.intercept_

        summary = pd.Series([features, accuracy, params, intercept], name='')
        models = models.append(summary)
        
        if ix%log == 0 and ix > 0:
            print(ix+1, 'feature sets analyzed')
            
    models.columns = ['features', 'accuracy', 'parameters', 'intercept']
    
    return models

def all_subsets(data_cols):
    return chain(*map(lambda x: combinations(data_cols, x), range(0, len(data_cols)+1)))

In [5]:
features = list(x_train.columns)
feature_sets = all_subsets(features)
models = compare_models(feature_sets, x_train, x_test, y_train, y_test)

101 feature sets analyzed
201 feature sets analyzed
301 feature sets analyzed
401 feature sets analyzed
501 feature sets analyzed


In [6]:
models = models.sort_values(by='accuracy', ascending=False).reset_index(drop=True)
models.head(10)

,features,accuracy,parameters,intercept
0,"[pclass, sex, age, sibsp, parch, Cherbourg, Qu...",0.790598,"[-0.713067526828976, -2.1712488949161717, -0.0...",3.573687
1,"[pclass, sex, age, sibsp, fare, Queenstown]",0.786325,"[-0.6126145810389865, -2.1245286369749707, -0....",3.107392
2,"[pclass, sex, age, sibsp, Cherbourg]",0.786325,"[-0.7425339391211495, -2.1527058397727052, -0....",3.426987
3,"[pclass, sex, age, sibsp, Queenstown]",0.786325,"[-0.7372057184424117, -2.136016184409732, -0.0...",3.526239
4,"[pclass, sex, age, sibsp, parch, fare, Queenst...",0.786325,"[-0.5916539450359095, -2.1697502925803436, -0....",3.093395
5,"[pclass, sex, age, sibsp, parch]",0.782051,"[-0.745857385003808, -2.1774318980909246, -0.0...",3.471734
6,"[pclass, sex, age, sibsp, parch, fare]",0.782051,"[-0.5944809017755814, -2.182362329972832, -0.0...",3.028571
7,"[pclass, sex, age, sibsp, parch, Cherbourg]",0.782051,"[-0.7399297988257098, -2.183256703513817, -0.0...",3.467405
8,"[pclass, sex, age, sibsp, parch, Queenstown]",0.782051,"[-0.7357644101484302, -2.158429265185262, -0.0...",3.550321
9,"[pclass, sex, age, sibsp, parch, fare, alone]",0.782051,"[-0.5322160040312065, -2.1146204335890126, -0....",3.299278


It seems the models with the highest accuracy on the test set make use of many features and have accuracies of nearly 80%.

If we were performing multiple iterations of model selection, feature selection and parameter tuning, then it would be advised to split the data into training, testing and __validation__ sets, where the validation sets would be used for the model selection. This would prevent the chosen model from being biased in favor of the test data.

However, since we are not doing this, and are simply comparing the accuracy of out-of-the-box logistic regression models on different sets of features, it is fine to use only a train-test split.

## Feature Crosses

I will make new features which are all the crosses of two features. For the purposes of this assignment, I will not consider higher-degree feature crosses. According to the `sklearn` documentation, "high degrees can cause overfitting."

Also, I will remove the feature crosses between my one-hot dummy variables which encode the same feature.

[`PolynomialFeatures`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) from `sklearn` can produce feature crosses quite easily. By default, it also includes squared individual features. I will include these. Overall, I expect 54 features before removing the redundant one-hot crosses. This is because I will have 9 original features, 9 squared features, and $9C2$ or 36 feature crosses.

In [7]:
x_train.head()

,pclass,sex,age,sibsp,parch,fare,alone,Cherbourg,Queenstown
446,1,0,25.0,1,2,151.5500,0,0,1
281,1,0,50.0,0,1,247.5208,0,0,0
536,1,1,35.0,0,0,26.5500,1,0,0
646,1,0,29.0,0,0,211.3375,1,0,1
224,2,1,19.0,0,0,10.5000,1,0,1


In [8]:
poly = PolynomialFeatures(2)
poly.fit(x_train)

cols = poly.get_feature_names(list(x_train.columns))
cross_train = poly.transform(x_train)
len(cols)

55

It seems there is one more feature than expected. I will print the feature names determined by the function to see if there is an index feature included.

In [9]:
cols[:10]

['1',
 'pclass',
 'sex',
 'age',
 'sibsp',
 'parch',
 'fare',
 'alone',
 'Cherbourg',
 'Queenstown']

The first feature name is not the name of any feature in the original data. I will inspect it to see if it is indeed just an index or a dummy feature, and if so remove it.

In [10]:
cross_train[:, 0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [0]:
cross_train = cross_train[:, 1:]
cross_test = poly.fit_transform(x_test)[:, 1:]

cross_train = pd.DataFrame(cross_train, columns=cols[1:])
cross_test = pd.DataFrame(cross_test, columns=cols[1:])

cross_train = cross_train.drop(['Cherbourg Queenstown', 'Cherbourg^2', 'Queenstown^2'], axis=1)
cross_test = cross_test.drop(['Cherbourg Queenstown', 'Cherbourg^2', 'Queenstown^2'], axis=1)

In [12]:
cross_train.head()

,pclass,sex,age,sibsp,parch,fare,alone,Cherbourg,Queenstown,pclass^2,...,parch alone,parch Cherbourg,parch Queenstown,fare^2,fare alone,fare Cherbourg,fare Queenstown,alone^2,alone Cherbourg,alone Queenstown
0,1.0,0.0,25.0,1.0,2.0,151.5500,0.0,0.0,1.0,1.0,...,0.0,0.0,2.0,22967.402500,0.0000,0.0,151.5500,0.0,0.0,0.0
1,1.0,0.0,50.0,0.0,1.0,247.5208,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,61266.546433,0.0000,0.0,0.0000,0.0,0.0,0.0
2,1.0,1.0,35.0,0.0,0.0,26.5500,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,704.902500,26.5500,0.0,0.0000,1.0,0.0,0.0
3,1.0,0.0,29.0,0.0,0.0,211.3375,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,44663.538906,211.3375,0.0,211.3375,1.0,0.0,1.0
4,2.0,1.0,19.0,0.0,0.0,10.5000,1.0,0.0,1.0,4.0,...,0.0,0.0,0.0,110.250000,10.5000,0.0,10.5000,1.0,0.0,1.0


## Comparing Models on Sets of Features Including Feature Crosses

It is infeasible to use brute force to compare all possible sets of features. The number of elements in the powerset of a set with 51 elements in it would be $\sum_{n=1}^{54}54Cn = 2251799813685247$

Since the set of features `(pclass, sex, age, sibsp, parch)` obtained reasonably high accuracy, I will consider all sets of features containing these 5 features, plus one feature cross.



In [13]:
base_features = ['pclass', 'sex', 'age', 'sibsp', 'parch']
feature_sets = [base_features + [cross_feature] for cross_feature in cross_train.columns[9:]]

feature_sets[:10]

[['pclass', 'sex', 'age', 'sibsp', 'parch', 'pclass^2'],
 ['pclass', 'sex', 'age', 'sibsp', 'parch', 'pclass sex'],
 ['pclass', 'sex', 'age', 'sibsp', 'parch', 'pclass age'],
 ['pclass', 'sex', 'age', 'sibsp', 'parch', 'pclass sibsp'],
 ['pclass', 'sex', 'age', 'sibsp', 'parch', 'pclass parch'],
 ['pclass', 'sex', 'age', 'sibsp', 'parch', 'pclass fare'],
 ['pclass', 'sex', 'age', 'sibsp', 'parch', 'pclass alone'],
 ['pclass', 'sex', 'age', 'sibsp', 'parch', 'pclass Cherbourg'],
 ['pclass', 'sex', 'age', 'sibsp', 'parch', 'pclass Queenstown'],
 ['pclass', 'sex', 'age', 'sibsp', 'parch', 'sex^2']]

In [0]:
models = compare_models(feature_sets, cross_train, cross_test, y_train, y_test)

In [15]:
models = models.sort_values(by='accuracy', ascending=False).reset_index(drop=True)
models.head(10)

,features,accuracy,parameters,intercept
0,"[pclass, sex, age, sibsp, parch, pclass^2]",0.803419,"[0.9074074445670642, -2.220952842244613, -0.02...",2.372148
1,"[pclass, sex, age, sibsp, parch, sex age]",0.799145,"[-0.8189312197026689, -0.6604917196082462, 0.0...",2.840853
2,"[pclass, sex, age, sibsp, parch, age parch]",0.794872,"[-0.7270558227129205, -2.2117800075740544, -0....",3.116795
3,"[pclass, sex, age, sibsp, parch, age sibsp]",0.794872,"[-0.7352323192351139, -2.1483417048369873, -0....",3.614215
4,"[pclass, sex, age, sibsp, parch, parch^2]",0.790598,"[-0.7158197676048329, -2.179679711993146, -0.0...",3.270578
5,"[pclass, sex, age, sibsp, parch, sex Queenstown]",0.790598,"[-0.7422410473492815, -2.0000232910095095, -0....",3.479592
6,"[pclass, sex, age, sibsp, parch, age fare]",0.786325,"[-0.6002038472195966, -2.173033571588067, -0.0...",3.205745
7,"[pclass, sex, age, sibsp, parch, pclass age]",0.786325,"[-0.2313549002905987, -2.2342086559381698, 0.0...",2.575101
8,"[pclass, sex, age, sibsp, parch, sibsp Cherbourg]",0.786325,"[-0.748261845467631, -2.179269153383214, -0.02...",3.477617
9,"[pclass, sex, age, sibsp, parch, pclass fare]",0.786325,"[-0.668773250263533, -2.183182073261965, -0.02...",3.144014


Using feature crosses resulted in higher accuracies. This hints at the potential gains that feature engineering can bring about, and there likely exists a even better set of features, potentially including multiple feature crosses, than what was found in this assignment.